<a href="https://colab.research.google.com/github/suarezthura1/Githubtesting/blob/main/Client.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
{
    #For the purpose of formatting, the formart function is used here
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "Project2.ipynb",  #This module defines our project
      "provenance": [],
      "collapsed_sections": []
    },
    "kernelspec": {   #List of arguments for our project starts here, for example kernel_name etc
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": { #Defines the scripting
      "name": "python"
    }
  },
  "cells": [  # %list_cells
    {
      "cell_type": "code",
      "execution_count": 2,
      "metadata": {
        "id": "FrsX7ycVxemp"
      },
      "outputs": [], 
      "source": [              #Dependencies; libraries are imported here
        "#Import packages. Feel free to add any others you might use!\n",
        "import itertools\n",
        "import jax.numpy as jnp\n",
        "import matplotlib.pyplot as plt\n",
        "from jax import random as jrandom\n",
        "from jax import nn as jnn\n",
        "import numpy as np\n",
        "import random\n",
        "import time\n",
        "import math"
      ]
    },
    {
      "cell_type": "markdown",
      "source": [  ##Each player is assigned unique identifier, i.e id
        "# Problem 1 - Optimal Strategy in 2 Player Pig"
      ],
      "metadata": {
        "id": "t8YlcbPP_W97"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 1a Solution"
      ],
      "metadata": {
        "id": "9HdtXVUh-YoL"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "State space $\\mathcal{S} = \\{r,s,t\\} $\n",  #Player's i.e runner's location and their respective action for every step of the play
        "where :\n",
        "\n",
        "$ r = \\text{ Runner location } \\in \\{0, 1, 2, \\ldots, N_{max}\\}$\n", 
        "\n",
        "$ s = \\text{ Spaces remaining for player 1 } \\in \\{0, 1, 2, \\ldots, N_{max}\\}$\n",
        "\n",
        "$ t = \\text{ Spaces remaining for player 2 } \\in \\{0, 1, 2,\\ldots, N_{max}\\}$\n",
        "\n",
        "---\n",
        "\n",
        "Actions $\\mathcal{A} = \\{\\text{Roll again}, \\text{Stay} \\}$\n",
        "\n",
        "---\n",
        "\n",
        "Rewards $R_s = \\begin{cases} 1\\text{ if r >= s (player 1 wins by holding)} \\\\ 0\\text{ otherwise} \\end{cases}$\n",
        "\n",
        "---\n",
        "\n",
        "Transition matrix when $N_{max} = 9$, $P = \\left[\\begin{array}{ccccc}\n", #Array container;multidimensional array for the two players and their ids, actions and location
        "0 & 0 & 0.2 & 0.2 & 0.2 & 0.2 & 0.2 & 0 & 0 & 0\\\\\n",                  #Encoding applies in this module
        "0 & 0 & 0 & 0.2 & 0.2 & 0.2 & 0.2 & 0.2 & 0 & 0\\\\\n",
        "0 & 0 & 0 & 0 & 0.2 & 0.2 & 0.2 & 0.2 & 0.2 & 0\\\\\n",
        "0 & 0 & 0 & 0 & 0 & 0.2 & 0.2 & 0.2 & 0.2 & 0.2\\\\\n",
        "0 & 0 & 0 & 0 & 0 & 0 & 0.2 & 0.2 & 0.2 & 0.4\\\\\n",
        "0 & 0 & 0 & 0 & 0 & 0 & 0 & 0.2 & 0.2 & 0.6\\\\\n",
        "0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0.2 & 0.8\\\\\n",
        "0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\\\\n",
        "0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\\\\n",
        "0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\n",
        "\\end{array}\\right]$\n",
        "\n",
        "This transition matrix can be extended for any $N_{max} > 0$ and follows the same general pattern"
      ],
      "metadata": {
        "id": "kEZefrSXuW5y"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 1b Solution"
      ],
      "metadata": {
        "id": "3amgJm0Y-aOe"
      }
    },
    {
      "cell_type": "markdown", #This module shows the probability of a player winning against his enemy
      "source": [
        "The probability of player 1 winning when they hold or bust is the probability that player 2 does not win on the next turn \n",
        "\n",
        "\n",
        "$P_{i,j,k} = max (P_{i,j,k,roll}, P_{i,j,k,hold})$\n",
        "\n",
        "$P_{i,j,k,roll} = 1/6[(1 − P_{0,k,j}) + P_{i+2,j,k} + P_{i+3,j,k} + P_{i+4,j,k} + P_{i+5,j,k} + P_{i+6,j,k}]$\n",
        "\n",
        "$P_{i,j,k,hold} = 1 − P_{0,k,j-i}$"
      ],
      "metadata": {
        "id": "2T7-5rupIjRu"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 1c Solution\n"
      ],
      "metadata": {
        "id": "Uk0il-YR-bto"
      }
    },
    {
      "cell_type": "code",   #Viusualization of the expected results when a play his made betwen player 1 and player 2
      "source": [
        "def visualizeMatrix(M): #visualize a matrix as a plot including the scale\n",
        "  fig = plt.figure()\n",
        "  ax = fig.add_subplot(111)\n",
        "  cax = ax.matshow(M, interpolation='nearest')\n",
        "  fig.colorbar(cax)\n",
        "  plt.gca().invert_yaxis()\n",
        "\n",
        "  plt.show()\n",
        "  return"
      ],
      "metadata": {
        "id": "_0Y4iYKLT592"
      },
      "execution_count": 8,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [  #movements of the players displayed in a simulation
        "def PIG_matrix(n):\n",
        "  #Matrix of movement transition probabilities matrix\n",
        "  #for the runner locations in PIG on a reroll\n",
        "  # -Assumes there is no bust (i.e. assuming you advance!)\n",
        "  # -Capped at n (so you can't exceed runners past n) for the pruposes of simulation\n",
        "  P = np.zeros((n + 1, n + 1))\n",
        "  i,j = np.indices(P.shape)\n",
        "  one_fifth = 1.0/5.0\n",
        "\n",
        "  P[i + 2 == j] = one_fifth \n",
        "  P[i + 3 == j] = one_fifth\n",
        "  P[i + 4 == j] = one_fifth\n",
        "  P[i + 5 == j] = one_fifth\n",
        "  P[i + 6 == j] = one_fifth\n",
        "\n",
        "  #Make the last column larger so that each row sums to 1\n",
        "  #This is effectively \"rounding down\" anything that goes over N_max \n",
        "  row_sums = np.sum(P, axis = 1)\n",
        "  P[:, n] += 1 - row_sums\n",
        "\n",
        "  return (P)"
      ],
      "metadata": {
        "id": "fYAuuZDDfyDa"
      },
      "execution_count": 9,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [ #chances of a player winning
        "def init_value_funcs_target(n):\n",
        "  # Value of staying. This is exactly correct!\n",
        "  q_stay = np.zeros((n + 1, n + 1, n + 1))\n",
        "  r,s,t = np.indices(q_stay.shape)\n",
        "  # If your runner is larger than the target you win by staying!\n",
        "  q_stay[r >= s] = 1 # If your runner is larger than the target you win by staying!\n",
        "  # otherwise the matrix will have all zeros to start and we will fill in with value iteration\n",
        "  q_stay[s == 0] = 0 # If player 1 has already won, no rewards\n",
        "  q_stay[t == 0] = 0 # If player 2 has already won, no rewards\n",
        "\n",
        "  # Value of rolling again.\n",
        "  # We intialize it to all zeros since the value is at least 0\n",
        "  # and we'll work our way up to the true value\n",
        "  q_roll_again = np.zeros((n + 1, n + 1, n + 1))\n",
        "\n",
        "  #Optimal value function is the maximum of staying or rolling again \n",
        "  v_star = np.maximum(q_stay, q_roll_again)\n",
        "\n",
        "  return q_stay, q_roll_again, v_star"
      ],
      "metadata": {
        "id": "hCY24MNagpdh"
      },
      "execution_count": 10,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [ #probability of a player winning against his enemy from a given position for first,second and third entries respectively
        "def optimal_2_player_pig(N_max, num_iters = 1):\n",
        "  '''Computes the optimal strategy in 2 player PIG for games up to size N_max'''\n",
        "  #Input: \n",
        "  #   N_max = Maximum distance that players can ever be from the finish line\n",
        "  #Output:\n",
        "  # v_star, roll_again\n",
        "  # v_star = An array of shape N_max,N_max,N_max with the probabability to win from a given position\n",
        "  # roll_again = An array of shape N_max,N_max,N_max with whether or not to roll again from a given position\n",
        "  #Note:\n",
        "  # The indexing for the state space should be \n",
        "  # First entry = How far the runner is advanced\n",
        "  # Second entry = Squares Remaining for Player 1\n",
        "  # Third entry = Squares Remaining for Player 2\n",
        "  # We assume it is player 1's turn. So the game starts at the position (N_max, N_max, 0)\n",
        "  # (r, p1, p2) represents the decision Player 1 has to make when\n",
        "  # Player 1 has p1 squares remaining, Player 2 has p2 squares remaining and the runner has advanced r squares  \n",
        "\n",
        "  #v_star = jnp.zeros((N_max, N_max, N_max))\n",
        "  #roll_again = jnp.zeros((N_max, N_max, N_max), dtype = bool)\n",
        "\n",
        "  n = N_max\n",
        "  prob_bust = 1.0/6.0\n",
        "\n",
        "  '''Does value iteration for PIG using the target objecting'''\n",
        "  q_stay, q_roll_again, v_star = init_value_funcs_target(n)\n",
        "  P = PIG_matrix(n)\n",
        "\n",
        "  for i in range(num_iters):\n",
        "    # Where j is the sum of squares remaining for player 1 (r) and player 2 (t)\n",
        "    #Only perform calculations when neither player has won the game
        "    for j in np.arange(2, n * 2 + 1):\n",
        "      for s in np.arange(1, n + 1):\n",
        "        t = j - s\n",
        "    \n",
        "        if t > 0 and t <= n:\n",
        "          for r in np.arange(n, -1, -1):\n",
        "            # Update acording to the Bellman Eqn!\n",
        "            q_roll_again[r, s, t] = prob_bust * (1 - v_star[0, t, s]) + (1 - prob_bust) * np.einsum('i,i', v_star[:, s, t], P[r, :])\n",
        "            \n",
        "            # If player 2 has not won, then choosing to stay will advance your position by r\n",
        "            if s > r:\n",
        "              # Probability that player 2 does not win on the next turn\n",
        "              q_stay[r, s, t] = 1 - v_star[0, t, s - r]\n",
        "              \n",
        "            # v_star is the maximum over all possible actions               \n",
        "            v_star[r, s, t] = np.maximum(q_stay[r, s, t], q_roll_again[r, s, t])\n",
        "\n",
        "  roll_again = np.greater(q_roll_again, q_stay)\n",
        "  \n",
        "  return v_star, roll_again\n",
        "\n",
        "# Note that 13 iterations gives a clear picture but requires 2 minutes to compute\n",
        "v_star, roll_again = optimal_2_player_pig(100, 13)"
      ],
      "metadata": {
        "id": "c3bPt4Bx-jG1"
      },
      "execution_count": 11,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "visualizeMatrix(roll_again[:,:,70])"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/", #flask url link that displays the outcome
          "height": 271
        },
        "id": "FOMZpmUDNtmm",
        "outputId": "0d9bc38e-9ec4-4aa0-d84e-fd57b67da821"
      },
      "execution_count": 12,
      "outputs": [
        {
          "output_type": "display_data",
          "data": { #source image, the location url
            "image/png": "iVBORw0KGgoAAAANSUhEUgAAATQAAAD+CAYAAAC0hfn5AAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4yLjIsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+WH4yJAAAU0ElEQVR4nO3df7AdZX3H8feHGyEF5UeMw8QklnSM2tRWcTKIg1OpYA3UEWfKUKK2aNNmnIq/W4Xa0Zb6h7ZWpTPU8QooOlbA6EhGU6MijNOORkJxlCSiaRRIBAEFdWyV5N5P/9hNPF5v7tl7ftyz57mfF7OTs3v27H7vkvvN93meffbINhERJThm1AFERAxKElpEFCMJLSKKkYQWEcVIQouIYiShRUQxWpHQJG2QdJekvZIuG3U8nSStlnSLpN2Sdkl6Xb19maQvSPpO/ecpo44VQNKEpDskfaZeXyNpR31tb5B07KhjPEzSyZK2SPqWpD2SntPi6/qG+v//nZI+LmlpW66tpGslPSDpzo5ts15HVf61jvkbkp41ipiHZeQJTdIEcBVwHrAO2Chp3Wij+hWHgDfZXgecCby6ju8y4Gbba4Gb6/U2eB2wp2P9XcB7bT8ZeBjYNJKoZncl8DnbTwOeQRV3666rpJXAa4H1tp8OTAAX055r+2Fgw4xtR7uO5wFr62Uz8P4FinFh2B7pAjwH2N6xfjlw+ajjmiPem4AXAHcBK+ptK4C7WhDbKqq/vM8HPgMIeAhYMtu1HnGsJwHfBTRjexuv60rgXmAZsKS+ti9s07UFTgPu7HYdgQ8AG2fbr4Rl5BUav/zLctj+elvrSDoNOB3YAZxq+776rfuBU0cUVqf3AW8Gpuv1xwOP2D5Ur7fp2q4BHgQ+VDeRr5Z0Ai28rrYPAO8G7gHuA34M3E57ry0c/TqOze9bL9qQ0MaCpMcCnwReb/snne+5+qdupHPIJL0IeMD27aOMYx6WAM8C3m/7dOBnzGhetuG6AtT9TxdQJeEnAifw60281mrLdVwIbUhoB4DVHeur6m2tIekxVMnsY7Y/VW/+gaQV9fsrgAdGFV/tLODFkr4HXE/V7LwSOFnSknqfNl3b/cB+2zvq9S1UCa5t1xXgXOC7th+0fRD4FNX1buu1haNfx9b/vvWjDQntNmBtPWJ0LFVn69YRx3SEJAHXAHtsv6fjra3AJfXrS6j61kbG9uW2V9k+jeoafsn2y4BbgAvr3UYe52G27wfulfTUetM5wG5adl1r9wBnSjq+/vtwONZWXtva0a7jVuDP6tHOM4EfdzRNx9+oO/HqjsnzgW8D/wO8ddTxzIjtuVTl+jeAr9fL+VT9UzcD3wG+CCwbdawdMZ8NfKZ+/VvA14C9wCeA40YdX0eczwR21tf208Apbb2uwD8A3wLuBD4KHNeWawt8nKpv7yBV5bvpaNeRaqDoqvp37ZtUI7cjv76DWlT/kBERY68NTc6IiIFIQouIYiShRUQxktAiohitSWiSNo86hqYS63Ak1uFoY6yzTaif8X5Pk+i7JrQFnMnfuos+h8Q6HIl1ONoY64eZe7ZFT5Pom1Ros514cc7kj4iBsP1l4Edz7HIB8BFXvko1K2NFt+Mu6baD7S/Xk7Jnnuzs+vV1wK3AWzqDAL5aP+9qhbvciXysjvNSjudELRuLm+IS63Ak1uE4HOtPefgh20/o9Tgv/IMT/MMfTTXa9/Zv/GIX8POOTZO2J+dxuqNNop8zl3RNaEcx35n8vxZE3a7fDNUFf67O7zGUiGjii95ydz+f/+GPpvja9ic12ndixXd+bnt9P+frRa8J7QjbljTvf6nqbD0JjM2/dBGLmYHpI0+mGrqeJtH3Osq5KGfyRyxmxhz0VKNlAHqaRN9rQlucM/kjFrnphv91I+njwFeAp0raL2mTpFdJelW9yzZgH9XE/w8Cf9Ukvq5NzvrEZwPLJe0H3g68E7hR0ibgbuCijiDOr4P4X+CVTYKIiPYzZmpAD7OwvbHL+wZePd/jNhnlPNqJzxlUEBExHqZb/uDbvgcFImJxMDCVhBYRpUiFFhFFMHCw5Q+ETUKLiEaM0+SMiEIYptqdz5LQIqKZaqZAuyWhRURDYgqNOog5JaFFRCPVoEASWkQUoLoPLQktIgoxnQotIkqQCi0iimHEVHu+V2lWSWgR0VianBFRBCMe9cSow5hTElpENFLdWJsmZ0QUIoMCEVEEW0w5FVpEFGI6FVpElKAaFGh3ymh3dBHRGhkUiIiiTOU+tIgoQWYKRERRpjPKGRElqCanJ6FFRAGMOJipTxFRApvcWBsRpVBurI2IMphUaBFRkAwKREQRjPKAx4goQ/U1du1OGe2OLiJaJF80HBGFMJkpEBEFaXuF1le6lfQGSbsk3Snp45KWSlojaYekvZJukHTsoIKNiNGxxbSPabQ0IWmDpLvqXHHZLO8/SdItku6Q9A1J53c7Zs8JTdJK4LXAettPByaAi4F3Ae+1/WTgYWBTr+eIiPaoBgUmGi3dSJoArgLOA9YBGyWtm7Hb3wE32j6dKrf8W7fj9tsgXgL8hqQlwPHAfcDzgS31+9cBL+nzHBHRCtV3CjRZGjgD2Gt7n+1HgeuBC2bsY+DE+vVJwPe7HbTnPjTbByS9G7gH+D/g88DtwCO2D9W77QdWzvZ5SZuBzQBLOb7XMCJigVSDAo370JZL2tmxPml7smN9JXBvx/p+4NkzjvH3wOclvQY4ATi320l7TmiSTqHKqGuAR4BPABuafr7+4SYBTtQy9xpHRCycecwUeMj2+j5PtxH4sO1/kfQc4KOSnm57+mgf6GeU81zgu7YfBJD0KeAs4GRJS+oqbRVwoI9zRERLDHimwAFgdcf6bLliE3WRZPsrkpYCy4EHjnbQfvrQ7gHOlHS8JAHnALuBW4AL630uAW7q4xwR0SLTHNNoaeA2YG19V8SxVJ3+W2fscw9VXkHSbwNLgQfnOmg/fWg7JG0B/hs4BNxB1YT8LHC9pHfU267p9RwR0R42HJwezI21tg9JuhTYTnWHxLW2d0m6AthpeyvwJuCDkt5A1YX3Cttzdk/1dWOt7bcDb5+xeR/VCEZEFKRqcg5upoDtbcC2Gdve1vF6N1U3VmOZKRARjbV9pkASWkQ0Ms/bNkYiCS0iGhpsk3MYktAiorF8p0BEFKEa5czX2EVEAfII7ogoSpqcEVGEjHJGRFEyyhkRRbDFoSS0iChFmpwRUYT0oUVEUZLQIqIIuQ8tIoqS+9Aiogg2HBrQAx6HJQktIhpLkzMiipA+tIgoipPQIqIUGRSIiCLY6UOLiGKIqYxyRkQp0ocWEUXIXM6IKIerfrQ2S0KLiMYyyhkRRXAGBSKiJGlyRkQxMsoZEUWwk9AioiC5bSMiipE+tIgoghHTGeWMiFK0vECjr3Qr6WRJWyR9S9IeSc+RtEzSFyR9p/7zlEEFGxEjVA8KNFmakLRB0l2S9kq67Cj7XCRpt6Rdkv692zH7rR+vBD5n+2nAM4A9wGXAzbbXAjfX6xFRAjdcupA0AVwFnAesAzZKWjdjn7XA5cBZtn8HeH234/ac0CSdBPw+cA2A7UdtPwJcAFxX73Yd8JJezxER7TLACu0MYK/tfbYfBa6nyh2d/hK4yvbD1bn9QLeD9lOhrQEeBD4k6Q5JV0s6ATjV9n31PvcDp/ZxjohoCQPT02q0AMsl7exYNs843Erg3o71/fW2Tk8BniLpvyR9VdKGbjH2MyiwBHgW8BrbOyRdyYzmpW1LmrUArX/AzQBLOb6PMCJiQRhofh/aQ7bX93nGJcBa4GxgFfBlSb9btwRn1U+Fth/Yb3tHvb6FKsH9QNIKgPrPWctE25O219te/xiO6yOMiFgodrOlgQPA6o71VfW2TvuBrbYP2v4u8G2qBHdUPSc02/cD90p6ar3pHGA3sBW4pN52CXBTr+eIiJYZ0KAAcBuwVtIaSccCF1Pljk6fpqrOkLScqgm6b66D9nsf2muAj9UB7QNeSZUkb5S0CbgbuKjPc0REKzS/JaMb24ckXQpsByaAa23vknQFsNP21vq9P5S0G5gC/sb2D+c6bl8JzfbXgdnayef0c9yIaKkB3llrexuwbca2t3W8NvDGemkkMwUiohmDpzM5PSKKkYQWEaVo+WTOJLSIaC4JLSKKML8ba0ciCS0iGssDHiOiHBnljIhSzD4zuz2S0CKimebTmkYmCS0iGlIGBSKiIKnQIqIY06MOYG5JaBHRTO5Di4iSZJQzIsrR8oTW7q9BjoiYh1RoEdFYmpwRUQaTqU8RUZBUaBFRijQ5I6IcSWgRUYwktIgogZwmZ0SUJKOcEVGKVGgRUY4ktIgoQvrQIqIoSWgRUQq1/AGPedpGRBQjFVpENJcmZ0QUIYMCEVGUJLSIKEbLE1rfgwKSJiTdIekz9foaSTsk7ZV0g6Rj+w8zIkZNVKOcTZZRGcQo5+uAPR3r7wLea/vJwMPApgGcIyJGzb+coN5taULSBkl31cXPZXPs98eSLGl9t2P2ldAkrQL+CLi6XhfwfGBLvct1wEv6OUdEtIgbLl1ImgCuAs4D1gEbJa2bZb/HURVNO5qE12+F9j7gzfzy+5QfDzxi+1C9vh9Y2ec5ok/bv//1I0tEXwaU0IAzgL2299l+FLgeuGCW/f6RqtX38yYH7TmhSXoR8IDt23v8/GZJOyXtPMgveg0jIhbQPJqcyw//ftfL5hmHWgnc27H+a8WPpGcBq21/tml8/YxyngW8WNL5wFLgROBK4GRJS+oqbRVwYLYP254EJgFO1LKWj52Mp1RkMXDNf1Mfst21z+toJB0DvAd4xXw+13OFZvty26tsnwZcDHzJ9suAW4AL690uAW7q9RwR0SIe6CjnAWB1x/rM4udxwNOBWyV9DzgT2NptYGAY96G9Bbhe0juAO4BrhnCOmMPMyuyFT3zmiCKJ4gyuLXUbsFbSGqpEdjHw0iOnsX8MLD+8LulW4K9t75zroANJaLZvBW6tX++j6vCLiMIMauqT7UOSLgW2AxPAtbZ3SboC2Gl7ay/HzUyBgqQyi6EbYG+37W3Athnb3naUfc9ucswktIhopvktGSOThFaAVGaxEESethERBUlCi6HJfWax4JLQIqIYSWgxaEerzNJ3FkOVJ9ZGRFGS0CKiFG3/GrsktDEx1wBAmpqxUNLkjIgy5Mba6Fcqs2iVJLSIKEFmCkTPctNstJGm253RktAiopn0ocV8NanM0ncWo5ImZ0SUIwktmuhWmaUqizZIhRYR5UhCi7lkNDPGhjP1KSIKkfvQIqIsbndGS0Ibgfk0MzMYEG2SCi0iypAba6NTKrMYdxkUiIhiJKFFbs2IMpgMCkREOTIosIj1Upml7yxaLQktIkqQG2sXqVRmMWiD6IedWNHnAew84DEiCtLufJaENkgZzVw8Fuv/6zQ5I6IMBtLkjIhitDuf9Z7QJK0GPgKcSvVjTtq+UtIy4AbgNOB7wEW2H+4/1Hbqt+nR1sGAzp+rrTEOy2JtTjYxyCanpA3AlcAEcLXtd854/43AXwCHgAeBP7d991zHPKaPeA4Bb7K9DjgTeLWkdcBlwM221wI31+sRUQBNu9HS9TjSBHAVcB6wDthY549OdwDrbf8esAX4p27H7blCs30fcF/9+qeS9gArgQuAs+vdrgNuBd7S63naqrR/xcfp5xmnWIsy2KdtnAHstb0PQNL1VLlj95HT2bd07P9V4OXdDjqQPjRJpwGnAzuAU+tkB3A/VZN0ts9sBjYDLOX4QYQREUNU3VjbOKMtl7SzY33S9mTH+krg3o71/cCz5zjeJuA/up2074Qm6bHAJ4HX2/6JpCPv2bY0e6u7/uEmAU7UspZ3Nf7SIKqDNvRJLdS3TM08z3yOm0qshZo/beMh2+sHcUpJLwfWA8/rtm9fCU3SY6iS2cdsf6re/ANJK2zfJ2kF8EA/54iI9phHhdbNAWB1x/qqetuvnk86F3gr8Dzbv+h20H5GOQVcA+yx/Z6Ot7YClwDvrP+8qddztEkp1cIwfo75HLOU67goDbYP7TZgraQ1VInsYuClnTtIOh34ALDBdqPCqJ8K7SzgT4FvSjr8t/RvqRLZjZI2AXcDF/VxjohojcHN5bR9SNKlwHaq2zautb1L0hXATttbgX8GHgt8ou7Kusf2i+c6bj+jnP9J1U84m3N6PW7bDLKiWOi+s2FVQ6myFrEBPuDR9jZg24xtb+t4fe58j5mZAhHRTL5oeHyNcxUyiNjH+eePIcojuCOiGO3OZ0loM41zZTLOscd40HS725xJaBHRjJnPjbUjkYQWEY0ID/LG2qFIQmN8m2rjGneMsSS0iChGElp7LVSFM+gbalOZxUikDy0iSpJRzhYaxwpnHGOO0jhNzogohElCa5NxqnLGKdZYRNrd4lxcCS0i+pP70FpgnKqdcYo1FqEktIgogg1T7W5zFp3Q2lLtzPVFIW2JMaKRVGgRUYwktIgogoEBfafAsBSX0MahCTcOMUb8OoPThxYRJTAZFFgoqXoiFkD60CKiGElow5XKLGKhZHJ6RJTCQB4fNBypzCJGIBVaRJQhU58GLpVZxIgYnPvQIqIYmSkwGKnMIlogfWgRUQQ7o5wRUZBUaL1LMzOiTYynpkYdxJxandAiokUW6+ODJG0ArgQmgKttv3M+n09lFtFSLb9t45hBH1DSBHAVcB6wDtgoad2gzxMRC8uAp91oaULSBkl3Sdor6bJZ3j9O0g31+zskndbtmMOo0M4A9treVwd1PXABsLvbB1OZRbSYB/eAx47C5wXAfuA2SVttd+aJTcDDtp8s6WLgXcCfzHXcgVdowErg3o71/fW2XyFps6SdknYe5BdDCCMiBs1TU42WBo4UPrYfBQ4XPp0uAK6rX28BzpGkuQ46skEB25PAJICkB7/oLT+bWMFDo4pnnpZDYh2CxDoch2P9zX4O8lMe3v5Fb1necPelknZ2rE/Wv/OHzVb4PHvGMY7sY/uQpB8Dj2eO6z6MhHYAWN2xvqredlS2nyBpp+31Q4hn4BLrcCTW4RhUrLY3DCKeYRpGk/M2YK2kNZKOBS4Gtg7hPBExvpoUPkf2kbQEOAn44VwHHXhCs30IuBTYDuwBbrS9a9DniYix1qTw2QpcUr++EPiSPfdUhaH0odneBmyb58cmu+/SGol1OBLrcLQu1rpP7HDhMwFca3uXpCuAnba3AtcAH5W0F/gRVdKbk7okvIiIsTGMPrSIiJFIQouIYiShRUQxktAiohhJaBFRjCS0iChGElpEFOP/AakouLghRVUvAAAAAElFTkSuQmCC\n",
            "text/plain": [
              "<Figure size 432x288 with 2 Axes>"
            ]
          },
          "metadata": {
            "needs_background": "light"
          }
        }
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "# Problem 2 - Playing PIG against the enemy"
      ],
      "metadata": {
        "id": "bm3Z_F1UxkV3"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 2a Solution\n"
      ],
      "metadata": {
        "id": "R98KdkDkxv9a"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "Please note the win rate of our \"optimal\" player using the SARSA algorithm is quite low, about 40 %. The player tends to play better using an epsilon of 1 and just choosing actions at random. Either this is a problem in our implementation or the problem is that it would take too many iterations to simulate all the possible games. After all the size of the state space is 1030301 for a game with 100 squares. Too find the optimal strategy it would be ideal to simulate each possible game multiple times which of course is impossible with over a million states and only 100000 iterations. To run the number of required simulations could take days, which is why we have not elected to do so."
      ],
      "metadata": {
        "id": "jaNmNdQ8OJFh"
      }
    },
    {
      "cell_type": "code",
      "source": [ #An example of enemy outcome function
        "#An example enemy outcome function\n",
        "# You may wish to create your own! (Hint: What you did in Project 1 will make this easy!)\n",
        "def timid_enemy(hero_squares_remaining, enemy_squares_remaining):\n",
        "  '''An example of an enemy_outcome function'''\n",
        "  #Input: Squares remaining for the heror and the enemy\n",
        "  #Output: A random number that is \\leq enemy_square_remaining indicating the squares remaining for the enemy after their turn\n",
        "\n",
        "  #This timid enemy rolls the dice exactly once and always stays. (They never push their luck by rolling more than once)\n",
        "  dice_roll = random.randint(1,7)\n",
        "  if dice_roll == 1:\n",
        "    #Enemy busted! They do not advance\n",
        "    return enemy_squares_remaining\n",
        "  else:\n",
        "    #If they don't bust they advance by the amount on the dice\n",
        "    #If this would advance them past the finish line, they just go to 0 square remaining\n",
        "    return max(enemy_squares_remaining - dice_roll, 0)"
      ],
      "metadata": {
        "id": "8CB6FYEoAMxJ"
      },
      "execution_count": 3,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "def get_next_state(s, a, enemy_outcome, n):\n",
        "  if a: # roll again\n",
        "    dice_roll = random.randint(1,7)\n",
        "\n",
        "    if dice_roll == 1: # busted\n",
        "      # simulate player 2s turn\n",
        "      return np.array([0, s[1], enemy_outcome(s[1], s[2])])\n",
        "    else: # runner advances\n",
        "      return np.array([min(s[0] + dice_roll, n), s[1], s[2]])\n",
        "  else: # hold\n",
        "    # update remaining squares for player 1\n",
        "    s_1 = max(s[1] - s[0], 0)\n",
        "    # simulate player 2s turn\n",
        "    return np.array([0, s_1, enemy_outcome(s_1, s[2])])\n",
        "\n",
        "def epsilon_greedy(s, q, epsilon):\n",
        "  '''Return the action dertermined from the q function from current_state using the epsilon-greedy algorithm'''\n",
        "  # This is the line from the pseudocode that says \"Choose A from S using policy derived from Q (e.g. epsilond greedy)\"\n",
        "\n",
        "  # Hold when it will win you the game\n",
        "  if s[0] >= s[1]:\n",
        "    return 0\n",
        "\n",
        "  probability_epsilon_event = (random.random() < epsilon)\n",
        "  \n",
        "  if probability_epsilon_event:  \n",
        "    return random.randint(0, 1) # Purely random\n",
        "  else:\n",
        "    return np.argmax(q[:, s[0], s[1], s[2]]) # Purely greedy\n"
      ],
      "metadata": {
        "id": "LogPwEPlJGcc"
      },
      "execution_count": 4,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [ #probability of a player winning from a given position
        "def optimal_strategy_against_enemy(enemy_outcome, N_max, num_episodes = 1):\n",
        "  '''Computes the optimal strategy in 2 player PIG for games up to size N_max'''\n",
        "  #Input: \n",
        "  #   enemy_outcome = A function that simulates the enemies strategy\n",
        "  #                   (See timit_enemy for an example)\n",
        "  #   N_max = Maximum distance that players can ever be from the finish line\n",
        "  #Output:\n",
        "  # v_star, roll_again\n",
        "  # v_star = An array of shape N_max+1,N_max+1,N_max+1 with the probabability to win from a given position\n",
        "  # roll_again =An array of shape N_max+1,N_max+1,N_max+1 with whether or not to roll again from a given position\n",
        "  #Note:\n",
        "  # The indexing for the state space should be \n",
        "  # First entry = How far the runner is advanced\n",
        "  # Second entry = Squares Remaining for Player 1\n",
        "  # Third entry = Squares Remaining for Player 2 \n",
        "  # We assume it is player 1's turn. So the game starts at the position (0,N_max,N_max)\n",
        "  # (r,p1,p2) represents the decision Player 1 has to make when\n",
        "  # Player 1 have p1 squares remaining, Player 2 has p2 squares remaining and the runner has advanced r squares\n",
        "\n",
        "  n = N_max\n",
        "\n",
        "  v_star = np.zeros((n + 1, n + 1, n + 1))\n",
        "  roll_again = np.zeros((n + 1, n + 1, n + 1), dtype = bool)\n",
        "  # q_star = (action, r, p1, p2)\n",
        "  q_star = np.zeros((2, n + 1, n + 1, n + 1))\n",
        "  i, j, k, l = np.indices(q_star.shape)\n",
        "\n",
        "\n",
        "  # If your runner is larger than the target you win by staying!\n",
        "  q_star[np.logical_and(i == 0, j >= k)] = 1 #If your runner is larger than the target you win by staying!\n",
        "  # otherwise the matrix will have all zeros to start and we will fill in with value iteration\n",
        "  q_star[np.logical_and(i == 0, k == 0)] = 0 # If player 1 has already won, no rewards\n",
        "  q_star[np.logical_and(i == 0, l == 0)] = 0 # If player 2 has already won, no rewards\n",
        "\n",
        "  \n",
        "  #q_star(a,s) is the value function if you play action a from state s\n",
        "  # and then play optimally afterwards\n",
        "  # The s-th, a-th entry of q_star represents q_star(a,s)\n",
        "\n",
        "  wins = np.zeros(2)\n",
        "\n",
        "  for i in range(num_episodes):\n",
        "    epsilon = 100 / np.sqrt(i + 1)\n",
        "    # epsilon = 1\n",
        "    learning_rate = 1 / np.sqrt(i + 1)\n",
        "    # learning_rate = 1 / num_episodes\n",
        "\n",
        "    # s is the current state\n",
        "    s = np.array([0, random.randint(1, n), random.randint(1, n)])\n",
        "    # a is the current action\n",
        "    a = epsilon_greedy(s, q_star, epsilon)\n",
        "\n",
        "    # Loop while no one has won\n",
        "    while (s[1] > 0 and s[2] > 0):\n",
        "      # s1 is the next chosen state\n",
        "      # take action a from state s\n",
        "      s1 = get_next_state(s, a, enemy_outcome, n)\n",
        "      #reward = 0 # The reward is always 0 in the Gambler's problem except for the terminal state\n",
        "      # a1 is the next chosen action\n",
        "      a1 = epsilon_greedy(s1, q_star, epsilon)\n",
        "\n",
        "      q_star[a, s[0], s[1], s[2]] += learning_rate * (q_star[a1, s1[0], s1[1], s1[2]] - q_star[a, s[0], s[1], s[2]])\n",
        "\n",
        "      s = s1\n",
        "      a = a1\n",
        "\n",
        "    if s[1] == 0:\n",
        "      wins[0] += 1\n",
        "    else:\n",
        "      wins[1] += 1\n",
        "\n",
        "  print(wins)\n",
        "      \n", #indexing for the state space should be How far the runner is advanced, Squares Remaining for Player 1, Squares Remaining for Player 2
        "  v_star = np.max(q_star, axis = 0)\n",
        "  roll_again = np.greater(q_star[1], q_star[0])\n",
        "\n",
        "  #Note that the indexing for the state space should be How far the runner is advanced, Squares Remaining for Player 1, Squares Remaining for Player 2 \n",
        "  return v_star, roll_again\n",
        "\n",
        "optimal_strategy_against_enemy(timid_enemy, 100, 100000)"
      ],
      "metadata": {
        "id": "QYFmiaHUAntp",
        "colab": {
          "base_uri": "https://localhost:8080/" #flask link url
        },
        "outputId": "e27fba4f-3abd-41d5-bde8-14909fc7e5e7"
      },
      "execution_count": 6,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "[38740. 61260.]\n"
          ]
        },
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "(array([[[0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 0.31234279, 0.28492399, ..., 0.05914765,\n",
              "          0.05297454, 0.05191068],\n",
              "         [0.        , 0.30705131, 0.26566416, ..., 0.08240784,\n",
              "          0.03843193, 0.01883904],\n",
              "         ...,\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ]],\n",
              " \n",
              "        [[0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         ...,\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ]],\n",
              " \n",
              "        [[0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 0.92445123, 0.91708976, ..., 1.        ,\n",
              "          0.98935387, 0.98850434],\n",
              "         [0.        , 0.9536751 , 0.93294085, ..., 0.9733496 ,\n",
              "          1.        , 1.        ],\n",
              "         ...,\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ]],\n",
              " \n",
              "        ...,\n",
              " \n",
              "        [[0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         ...,\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ]],\n",
              " \n",
              "        [[0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         ...,\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         [0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ]],\n",
              " \n",
              "        [[0.        , 0.        , 0.        , ..., 0.        ,\n",
              "          0.        , 0.        ],\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         ...,\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ],\n",
              "         [0.        , 1.        , 1.        , ..., 1.        ,\n",
              "          1.        , 1.        ]]]),\n",
              " array([[[False, False, False, ..., False, False, False],\n",
              "         [False,  True,  True, ...,  True,  True,  True],\n",
              "         [False,  True,  True, ...,  True,  True,  True],\n",
              "         ...,\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False]],\n",
              " \n",
              "        [[False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         ...,\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False]],\n",
              " \n",
              "        [[False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         ...,\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False]],\n",
              " \n",
              "        ...,\n",
              " \n",
              "        [[False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         ...,\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False]],\n",
              " \n",
              "        [[False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         ...,\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False]],\n",
              " \n",
              "        [[False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         ...,\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False],\n",
              "         [False, False, False, ..., False, False, False]]]))"
            ]
          },
          "metadata": {},
          "execution_count": 6
        }
      ]
    },
    {
      "cell_type": "markdown",
      "source": [
        "## 2b Solution\n"
      ],
      "metadata": {
        "id": "_HndY1SNynyP"
      }
    },
    {
      "cell_type": "markdown",
      "source": [
        "Optimal strategy using the SARSA algorithm is dependent on the strategy used by the enemy. The optimal strategy in problem 1 is to play in such a way that will maximize the probability of player 1 winning in any given state, and assumes that player 2 is using the same optimal strategy. In problem 2 however, player 2 is free to use any strategy.  Different strategies will have different strengths and weaknesses, and will need to be countered differently for optimal play."
      ],
      "metadata": {
        "id": "o_C5oJMADPoJ"
      }
    }
  ]
}

{'cells': [{'cell_type': 'code',
   'execution_count': 2,
   'metadata': {'id': 'FrsX7ycVxemp'},
   'outputs': [],
   'source': ['#Import packages. Feel free to add any others you might use!\n',
    'import itertools\n',
    'import jax.numpy as jnp\n',
    'import matplotlib.pyplot as plt\n',
    'from jax import random as jrandom\n',
    'from jax import nn as jnn\n',
    'import numpy as np\n',
    'import random\n',
    'import time\n',
    'import math']},
  {'cell_type': 'markdown',
   'metadata': {'id': 't8YlcbPP_W97'},
   'source': ['# Problem 1 - Optimal Strategy in 2 Player Pig']},
  {'cell_type': 'markdown',
   'metadata': {'id': '9HdtXVUh-YoL'},
   'source': ['## 1a Solution']},
  {'cell_type': 'markdown',
   'metadata': {'id': 'kEZefrSXuW5y'},
   'source': ['State space $\\mathcal{S} = \\{r,s,t\\} $\n',
    'where :\n',
    '\n',
    '$ r = \\text{ Runner location } \\in \\{0, 1, 2, \\ldots, N_{max}\\}$\n',
    '\n',
    '$ s = \\text{ Spaces remaining for player 1 } \\in 